# 深層学習を使って手書き数字を認識する

# ライブラリ

### 深層学習用のライブラリ  
下記は深層学習を使って学習をするために必要なライブラリ。  
実際の学習はノートパソコンで行うには計算量が多く時間がかかるため、実行できない形式にしている。  
実行してみたい人は、セルのモードを`markdown`から`code`に変更し、　本文の上下についているコードインデント、` ``` `  を取り外せば実行できる。

```python
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
```

## 学習モデルのロードとデータ取得に必要なライブラリ
下記は本講義で使用するので実行すること

In [ ]:
from keras.datasets import mnist
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

windowsユーザーで、上記記の実行で固まってしまったりして先に進まない場合は`Anaconda Prompt`で次を実行
```bash
pip uninstall tensorflow
pip uninstall keras
conda install tensorflow=1.14.0
conda install keras
```
上記を実行して`jupyter`がうまく動かなくなってしまった場合は下記を実行
```bash
conda install jupyter
```
※`jupyter lab` を使用している方は下記も同時に実行
```bash
conda install jupyterlab
```
----

### 基本処理、画像描画用のライブラリ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import cv2
import os
from IPython.display import clear_output

その他、必要なライブラリ一覧
- matplotlib
- numpy
- pillow
- opencv-python

import cv2 などして 下記エラーが出た場合、該当するライブラリのインストールが必要となる
```python
ModuleNotFoundError: No module named 'cv2'
````
上記の場合は`opencv-python`が必要。　`jupyter` のセル上で下記コマンドを実行するとライブラリをインストールできる。
```
! pip install opencv-python
```
※↑`jupyter`のセル上でAnaconda Promptのコマンド、またはbashのコマンドを打つ場合は文頭に必ず`!`をつけること

# 学習データの準備

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## データの確認
手書き数字画像データ 訓練データの一番目の画像を確認

In [ ]:
plt.imshow(x_train[0], cmap="gray")
plt.xticks([]);
plt.yticks([]);
plt.savefig("predict_test.png")

訓練データ一番目の正解ラベルを調べる

In [ ]:
y_train[0]

訓練データ先頭の81個の画像を確認する

In [ ]:
fig = plt.figure(figsize=(9, 9))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)

for i in range(81):
    ax = fig.add_subplot(9, 9, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i].reshape(28,28), cmap="gray")

訓練データの正解ラベルの先頭81個を表示

In [ ]:
y_train[:81]

訓練データの正解ラベルのユニークな種別を表示

In [ ]:
import numpy as np

In [ ]:
classes = np.unique(y_train)
classes

画像から識別したいラベルの種類は10種類

In [ ]:
num_classes = len(classes)
num_classes

# 深層学習のための準備
---
ここから下は深層学習を実際に行う準備と、実際の学習実行のためのコードが続く。  
実際の学習はノートパソコンで行うには計算量が多く時間がかかるため、実行できない形式にしている。  
実行してみたい人は、セルのモードを`markdown`から`code`に変更し、　本文の上下についているコードインデント、` ``` `  を取り外せば実行できる。

```python
# input image dimensions
img_rows, img_cols = x_train[0].shape
# 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
 
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
```

出力：

```python
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
```

# 畳み込みニューラルネットワークの実装

```python
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
```

### モデルのサマリー

```python
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
=================================================================
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
_________________________________________________________________
```

# モデルのコンパイル

```python
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
```

# 学習データの訓練および検証

## 学習設定

```python
batch_size = 128
num_classes = 10
epochs = 12
```

## 学習の実行

```python
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
```

出力：

```python
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 16s 272us/step - loss: 0.2643 - acc: 0.9190 - val_loss: 0.0608 - val_acc: 0.9809
Epoch 2/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0915 - acc: 0.9732 - val_loss: 0.0414 - val_acc: 0.9856
Epoch 3/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0679 - acc: 0.9799 - val_loss: 0.0404 - val_acc: 0.9854
Epoch 4/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0563 - acc: 0.9829 - val_loss: 0.0345 - val_acc: 0.9878
Epoch 5/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0484 - acc: 0.9858 - val_loss: 0.0335 - val_acc: 0.9887
Epoch 6/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0420 - acc: 0.9869 - val_loss: 0.0296 - val_acc: 0.9899
Epoch 7/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0392 - acc: 0.9876 - val_loss: 0.0289 - val_acc: 0.9904
Epoch 8/12
60000/60000 [==============================] - 9s 148us/step - loss: 0.0358 - acc: 0.9889 - val_loss: 0.0279 - val_acc: 0.9906
Epoch 9/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0331 - acc: 0.9898 - val_loss: 0.0283 - val_acc: 0.9913
Epoch 10/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0308 - acc: 0.9907 - val_loss: 0.0293 - val_acc: 0.9911
Epoch 11/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0279 - acc: 0.9912 - val_loss: 0.0275 - val_acc: 0.9905
Epoch 12/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0283 - acc: 0.9917 - val_loss: 0.0267 - val_acc: 0.9915
Test loss: 0.026699552729436436
Test accuracy: 0.9915
```

# モデルの読み込み
上記学習により得られた **学習モデル** ファイル（h5形式）を読み込み、手書き数字の推論を行う。

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

## モデルの読み込み

In [ ]:
model=load_model('MNIST99.h5')

## 推論の実行

In [ ]:
# img_path = 'predict_test.png'
img_path = 'number01.png'
# 数字画像の読み込み
load_image = load_img(img_path, target_size=(28, 28), color_mode="grayscale")
img = img_to_array(load_image)
# 画像のプレビュー
plt.imshow(img.reshape(28,28), cmap="gray")
plt.xticks([])
plt.yticks([])

img_nad = img/255
img_nad = img_nad[None, ...]

# 推論ラベルの設定
label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# 推論の実行
pred = model.predict(img_nad, batch_size=1, verbose=0)

# 結果の出力
score = np.max(pred)
pred_label = label[np.argmax(pred[0])]
print('name:',pred_label)
print('score:',score)

## 推論システムを関数化

In [ ]:
def mnist_predict(img_path, invers=False):
    load_image = load_img(img_path, target_size=(28, 28), color_mode="grayscale")

    img = img_to_array(load_image)
    if invers:
        img = 255 - img
    plt.imshow(img.reshape(28,28), cmap="gray")
    plt.xticks([])
    plt.yticks([])

    img_nad = img/255
    img_nad = img_nad[None, ...]
    
    label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    pred = model.predict(img_nad, batch_size=1, verbose=0)
    score = np.max(pred)
    pred_label = label[np.argmax(pred[0])]
    print('name:',pred_label)
    print('score:',score)

### 関数の実行

In [ ]:
mnist_predict("number01.png")

# インカメラで取った画像を即座に推論

`p`キーを押して推論する。　色を反転させた画像での推論は`i`キー。終了は`q`キーを押す

In [ ]:
import cv2
import os
from IPython.display import clear_output

def predict_camera_capture(device_num, dir_path,
                          size=(200, 200), ext='png', 
                          delay=1, window_name='frame'):

    cap = cv2.VideoCapture(device_num)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if size is not None and len(size) == 2:
            frame = cv2.resize(frame, size)
        
        cv2.imshow(window_name, frame)
        key = cv2.waitKey(delay) & 0xFF

        if key == ord('p'):
            clear_output(wait=True)
            cv2.imwrite('predict.png', frame)
            mnist_predict('predict.png')
            plt.show()

        elif key == ord('i'):
            clear_output(wait=True)
            cv2.imwrite('predict.png', frame)
            mnist_predict('predict.png', invers=True)
            plt.show()

        elif key == ord('q'):
            os.remove("predict.png")
            break

    cv2.destroyWindow(window_name)

In [ ]:
predict_camera_capture(0, "capture_images")